In [2]:
from bs4 import BeautifulSoup
import requests

In [4]:
root_url = 'https://cooljugator.com'
home_url = '/de/list/all'
r = requests.get(root_url + home_url)
soup = BeautifulSoup(r.text, 'html.parser')

In [5]:
def create_verb_rec():
    pronouns = ['ich', 'du', 'er', 'wir', 'ihr', 'Sie']
    verb_rec = dict.fromkeys(['verb', 'translation', 'present', 'perfect', 'future'])
    for case in ['present', 'perfect', 'future']:
        verb_rec[case] = dict.fromkeys(pronouns)
    return verb_rec

In [90]:
verb_collection = []

for verb_link in soup.find_all('li', attrs={'class': 'item'}):
    verb_rec = create_verb_rec()
    verb, translation = verb_link.text.split(' - ')
    verb_rec['verb'] = verb
    verb_rec['translation'] = translation
    url_suffix = verb_link.find('a', href=True)['href']
    verb_url = root_url + url_suffix
    verb_soup = BeautifulSoup(requests.get(verb_url).text, 'html.parser')
    for case in ['present', 'perfect', 'future']:
        for i, pronoun in enumerate(pronouns):
            case_class = '{}{}'.format(case, i+1) 
            conjugated_verb = verb_soup.find('div', {"id": case_class}).find('div', {'class': 'meta-form'}).text
            verb_rec[case][pronoun] = conjugated_verb
    verb_collection.append(verb_rec)

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [92]:
len(verb_collection)

738

In [122]:
{('present', k):v for k, v in verb_collection[0]['present'].items()}

{('present', 'ich'): 'aale',
 ('present', 'du'): 'aalst',
 ('present', 'er'): 'aalt',
 ('present', 'wir'): 'aalen',
 ('present', 'ihr'): 'aalt',
 ('present', 'Sie'): 'aalen'}

In [125]:
{k: v for k, v in verb_collection[0].items()}

{'verb': 'aalen',
 'translation': 'stretch out',
 'present': {'ich': 'aale',
  'du': 'aalst',
  'er': 'aalt',
  'wir': 'aalen',
  'ihr': 'aalt',
  'Sie': 'aalen'},
 'past_perfect': None,
 'future': {'ich': 'werde aalen',
  'du': 'wirst aalen',
  'er': 'wird aalen',
  'wir': 'werden aalen',
  'ihr': 'werdet aalen',
  'Sie': 'werden aalen'},
 'perfect': {'ich': 'habe geaalt',
  'du': 'hast geaalt',
  'er': 'hat geaalt',
  'wir': 'haben geaalt',
  'ihr': 'habt geaalt',
  'Sie': 'haben geaalt'}}

In [149]:
x = []
for vc in verb_collection:
    d = {}
    for k, v in vc.items():
        if isinstance(v, str):
            d[k] = v
        else:
            for ki, vi in v.items():
                d[(k, ki)] = vi
        x.append(d)

In [152]:
pd.DataFrame.to_csv?

In [157]:
writer = pd.ExcelWriter('/Users/pto845/Documents/verbs.xlsx', options={'encoding':'utf-8'})
# Write the frame to excel
pd.DataFrame(x).to_excel(writer, 'Sheet1')
writer.save()

In [151]:
pd.DataFrame(x).to_csv('/Users/pto845/Documents/german_verbs.csv')

In [141]:
[{k:v if isinstance(v, str) else {(k, ki): vi for ki, vi in v.items()} for k, v in vc.items()} for vc in verb_collection][0]

{'verb': 'aalen',
 'translation': 'stretch out',
 'present': {('present', 'ich'): 'aale',
  ('present', 'du'): 'aalst',
  ('present', 'er'): 'aalt',
  ('present', 'wir'): 'aalen',
  ('present', 'ihr'): 'aalt',
  ('present', 'Sie'): 'aalen'},
 'future': {('future', 'ich'): 'werde aalen',
  ('future', 'du'): 'wirst aalen',
  ('future', 'er'): 'wird aalen',
  ('future', 'wir'): 'werden aalen',
  ('future', 'ihr'): 'werdet aalen',
  ('future', 'Sie'): 'werden aalen'},
 'perfect': {('perfect', 'ich'): 'habe geaalt',
  ('perfect', 'du'): 'hast geaalt',
  ('perfect', 'er'): 'hat geaalt',
  ('perfect', 'wir'): 'haben geaalt',
  ('perfect', 'ihr'): 'habt geaalt',
  ('perfect', 'Sie'): 'haben geaalt'}}

In [142]:
[(k, v) for k, v in verb_collection[0].items()]

[('verb', 'aalen'),
 ('translation', 'stretch out'),
 ('present',
  {'ich': 'aale',
   'du': 'aalst',
   'er': 'aalt',
   'wir': 'aalen',
   'ihr': 'aalt',
   'Sie': 'aalen'}),
 ('future',
  {'ich': 'werde aalen',
   'du': 'wirst aalen',
   'er': 'wird aalen',
   'wir': 'werden aalen',
   'ihr': 'werdet aalen',
   'Sie': 'werden aalen'}),
 ('perfect',
  {'ich': 'habe geaalt',
   'du': 'hast geaalt',
   'er': 'hat geaalt',
   'wir': 'haben geaalt',
   'ihr': 'habt geaalt',
   'Sie': 'haben geaalt'})]

In [ ]:
pd.DataFrame([])

In [118]:
verb_collection[0]

{'verb': 'aalen',
 'translation': 'stretch out',
 'present': {'ich': 'aale',
  'du': 'aalst',
  'er': 'aalt',
  'wir': 'aalen',
  'ihr': 'aalt',
  'Sie': 'aalen'},
 'past_perfect': None,
 'future': {'ich': 'werde aalen',
  'du': 'wirst aalen',
  'er': 'wird aalen',
  'wir': 'werden aalen',
  'ihr': 'werdet aalen',
  'Sie': 'werden aalen'},
 'perfect': {'ich': 'habe geaalt',
  'du': 'hast geaalt',
  'er': 'hat geaalt',
  'wir': 'haben geaalt',
  'ihr': 'habt geaalt',
  'Sie': 'haben geaalt'}}

In [117]:
pd.concat({
        k: pd.DataFrame.from_dict(v, 'index') for k, v in verb_collection[0].items() if isinstance(v, dict)
    }, 
    axis=0)

0
future  ich   werde aalen
        du    wirst aalen
        er     wird aalen
        wir  werden aalen
        ihr  werdet aalen
        Sie  werden aalen
perfect ich   habe geaalt
        du    hast geaalt
        er     hat geaalt
        wir  haben geaalt
        ihr   habt geaalt
        Sie  haben geaalt
present ich          aale
        du          aalst
        er           aalt
        wir         aalen
        ihr          aalt
        Sie         aalen

In [111]:
pd.Panel(verb_collection[0])

/Users/pto845/.virtualenvs/work/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)


KeyError: 1

In [95]:
import pandas as pd
pd.DataFrame(verb_collection[:5])

,future,past_perfect,perfect,present,translation,verb
0,"{'ich': 'werde aalen', 'du': 'wirst aalen', 'e...",None,"{'ich': 'habe geaalt', 'du': 'hast geaalt', 'e...","{'ich': 'aale', 'du': 'aalst', 'er': 'aalt', '...",stretch out,aalen
1,"{'ich': 'werde aasen', 'du': 'wirst aasen', 'e...",None,"{'ich': 'habe geaast', 'du': 'hast geaast', 'e...","{'ich': 'aase', 'du': 'aast', 'er': 'aast', 'w...",waste,aasen
2,"{'ich': 'werde abaasen', 'du': 'wirst abaasen'...",None,"{'ich': 'habe abgeaast', 'du': 'hast abgeaast'...","{'ich': 'aase ab', 'du': 'aast ab', 'er': 'aas...",do,abaasen
3,"{'ich': 'werde abacheln', 'du': 'wirst abachel...",None,"{'ich': 'habe abgeachelt', 'du': 'hast abgeach...","{'ich': 'achle ab', 'du': 'achelst ab', 'er': ...",do,abacheln
4,"{'ich': 'werde abaissieren', 'du': 'wirst abai...",None,"{'ich': 'habe abaissiert', 'du': 'hast abaissi...","{'ich': 'abaissiere', 'du': 'abaissierst', 'er...",do,abaissieren


In [94]:
verb_collection[-1]

{'verb': 'aufklaren',
 'translation': 'clear',
 'present': {'ich': 'klare auf',
  'du': 'klarst auf',
  'er': 'klart auf',
  'wir': 'klaren auf',
  'ihr': 'klart auf',
  'Sie': 'klaren auf'},
 'past_perfect': None,
 'future': {'ich': 'werde aufklaren',
  'du': 'wirst aufklaren',
  'er': 'wird aufklaren',
  'wir': 'werden aufklaren',
  'ihr': 'werdet aufklaren',
  'Sie': 'werden aufklaren'},
 'perfect': {'ich': 'habe aufgeklart',
  'du': 'hast aufgeklart',
  'er': 'hat aufgeklart',
  'wir': 'haben aufgeklart',
  'ihr': 'habt aufgeklart',
  'Sie': 'haben aufgeklart'}}